In [2]:
# Peri 8/3/24
# A basic Agent Based Model to understand its input/output requirements to be linked with other modules
# Behavioural and traffic microsimulation model
# Importing necessary libraries
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import imageio
from scipy.spatial import distance_matrix

# Defining Agent class
class Agent():
    def __init__(self,x,y,infected,resistance,infectedCounter = 0):
        # Initialize position and infection status of the agent
        self.x = x
        self.y = y
        self.infected = infected
        self.resistance = resistance
        self.infectedCounter = infectedCounter 
        
    def movement(self,stepSize,xBounds,yBounds):
        # Define movement for each agent within specified bounds
        self.x += stepSize*np.random.uniform(-1,1)
        self.y += stepSize*np.random.uniform(-1,1)
        
        # Ensure agents stay within bounds
        if self.x < xBounds[0]:
            self.x = xBounds[0]
        elif self.x > xBounds[1]:
            self.x = xBounds[1]
        
        if self.y < yBounds[0]:
            self.y = yBounds[0]
        elif self.y > yBounds[1]:
            self.y = yBounds[1]
            
        # Decrement infected counter
        if self.infected == True:
            self.infectedCounter -= 1
            if self.infectedCounter <= 0:
                self.infected = False
        
    def infect(self):
        # Spread infection to nearby agents based on resistance
        infectRoll = np.random.uniform()
        
        if self.infected == False and infectRoll > self.resistance:
            self.infected = True
            self.infectedCounter = 50
            self.resistance *= 1.5

# Function to get positions of all agents
def getPosition(agents):
    positions = []
    for agent in agents:
        positions.append([agent.x,agent.y])
    positions = np.array(positions)
    return positions

# Function to move all agents
def moveAgents(agents,stepSize,xBounds,yBounds):
    for i in range(0,len(agents)):
        agents[i].movement(stepSize,xBounds,yBounds)
    return agents

# Function to get infection status of all agents
def getInfected(agents):
    infected  = []
    for agent in agents:
        infected.append(agent.infected)
    return infected

# Function to find nearby agents for potential infection
def getCloseAgents(distanceMatrix,agentNumber):
    sort = np.argsort(distanceMatrix[agentNumber])
    closeMask = distanceMatrix[agentNumber][sort] < 10
    closeAgents = np.argsort(distanceMatrix[agentNumber])[closeMask][1:]
    return closeAgents

# Function to simulate infection spread
def rollInfect(agents):
    positions = getPosition(agents)
    distanceMatrix = distance_matrix(positions,positions)
    for i in range(0,len(agents)):
        closeAgents = getCloseAgents(distanceMatrix,i)
        for j in closeAgents:
            if agents[j].infected == True:
                agents[i].infect()
    return agents

# Function to create GIF of simulation frames
def makeGif(frames,name):
    !mkdir frames
    
    counter=0
    images = []
    for i in range(0,len(frames)):
        plt.figure(figsize = (6,6))
        plt.scatter(frames[i][0],frames[i][1], c = frames[i][2], cmap = "RdYlGn_r")
        plt.title("Infected = " + str(np.sum(frames[i][2])))
        plt.tick_params(left = False, right = False , labelleft = False,labelbottom = False, bottom = False)
        plt.savefig("frames/" + str(counter)+ ".png")
        images.append(imageio.imread("frames/" + str(counter)+ ".png"))
        counter += 1
        plt.close()
        
    imageio.mimsave(name, images)
        
    !rm -r frames

# Initializing agents with random positions and resistance values
agents = []
for i in range(0,500):
    x = np.random.uniform(0,500)
    y = np.random.uniform(0,500)
    agents.append(Agent(x,y,0,.3))
    
# Infecting a few initial agents
for i in range(0,10):
    agents[i].infected = 1
    agents[i].infectedCounter = 50
    
# Setting simulation parameters
stepSize = 5
xBounds = [0,500]
yBounds = [0,500]

# Simulating movement and infection spread, storing frames
frames = []
for i in range(0,500):
    agents = moveAgents(agents,stepSize,xBounds,yBounds)
    agents = rollInfect(agents)
    positions = getPosition(agents)
    infected = getInfected(agents)
    frames.append([positions[:,0] , positions[:,1], infected])

# Creating GIF from simulation frames
makeGif(frames,"Simulation.gif")


C:\Users\kontoroupisp\AppData\Local\Temp\ipykernel_22816\1999950119.py:103: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread("frames/" + str(counter)+ ".png"))
'rm' is not recognized as an internal or external command,
operable program or batch file.
